In [2]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
import numpy as np
import gensim
from gensim.models import Word2Vec
from nltk.data import find
from nltk.corpus import wordnet as wn
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import random

data = pd.read_csv('./all_cc_jokes.csv', sep = ',', index_col = 0, names = ['type', 'link', 'text'])
word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
model = gensim.models.Word2Vec.load_word2vec_format(word2vec_sample, binary=False)
tfidfvectorizer = TfidfVectorizer(max_features=2000, stop_words='english')
tfidf_counts = tfidfvectorizer.fit_transform(data['text'].values.astype('U'))

In [844]:
data = pd.read_csv('./all_cc_jokes.csv', sep = ',', index_col = 0, names = ['type', 'link', 'text'])


In [3]:
def latent_topic(joke_index, tfidf_counts, tfidfvectorizer, model):
    """creates a latent topic for a joke using an existing word2vec model
    does so by doing a weighted avg of word2vec """
    print('\rdoing joke {}'.format(joke_index), end='')
    scores = tfidf_counts[joke_index]
    result = None
    for j, index in enumerate(scores.indices):
        curr_word = tfidfvectorizer.get_feature_names()[index]
        try:
            if result is None:
                result = model[curr_word]
            else:
                result += model[curr_word]
        except:
            pass
    return result

def latent_topic3(joke_index, tfidf_counts, tfidfvectorizer, model):
    """creates a latent topic for a joke using an existing word2vec model
    does so by doing a weighted avg of word2vec """
    print('\rdoing joke {}'.format(joke_index), end='')
    scores = tfidf_counts[joke_index]
    result = None
    for j, index in enumerate(scores.indices):
        curr_word = tfidfvectorizer.get_feature_names()[index]
        try:
            if result is None:
                result = model[curr_word] * scores.data[j]
            else:
                result += model[curr_word] *scores.data[j]
        except:
            pass
    if result is not None:
        return result/len(scores.indices)
    else:
        return

def latent_topic2(joke_index, tfidf_counts, tfidfvectorizer, model):
    """creates a latent topic for a joke using an existing word2vec model
    does so by doing a weighted avg of word2vec """
    print('\rdoing joke {}'.format(joke_index), end='')
    scores = tfidf_counts[joke_index]
    result = None
    for index in scores.indices:
        curr_word = tfidfvectorizer.get_feature_names()[index]
        try:
            if result is None:
                result = model[curr_word].reshape((1,300))
            else:
                result = np.append(result,model[curr_word].reshape((1,300)), axis=0)
        except:
            pass
    if result is None:
        return None
    print(result.shape)
    mean = np.mean(result, axis=0)
    std = np.std(result, axis=0)
    return np.append(mean-std, mean+std, axis = 0).reshape((600))
result = latent_topic(20, tfidf_counts, tfidfvectorizer, model)


doing joke 20

In [4]:
generate = False
try:
    all_latent_topics = np.load('./docvec.npy')
    print("successfully loaded latent topics for jokes from file")
except:
    generate = True

if generate:
    print("generating latent topics for jokes file,")
    all_latent_topics = np.zeros((data.shape[0],300))
    for i in range(data.shape[0]):
        result = latent_topic(i, tfidf_counts, tfidfvectorizer, model)
        if result is not None:
            all_latent_topics[i] = result 
    print("\ndone! saved as latent_topics.npy")
    np.save('./latent_topics.npy', all_latent_topics)

successfully loaded latent topics for jokes from file


In [196]:
def find_most_similar(index, latent_topics, jokes):
    curr_topic = latent_topics[index]
    ssd = np.sum(np.square(latent_topics - curr_topic), axis=1)
    top = 0
    index = np.argsort(ssd)[top]
    print("how many equal 0", np.sum(ssd == 0))

    while ssd[index] == 0 and top < 8:
#         print ("skip, identical")
#         print ('index', index)
#         print ('actual', np.sum(np.square(latent_topics[index] - curr_topic)))
#         print (latent_topics[index][:10])
#         print(curr_topic[:10])
#         print ('ssd', ssd[index])
#         print (jokes[index])
        top += 1
        index = np.argsort(ssd)[top]

#     print (np.sum(np.square(latent_topics[np.argsort(ssd)[index]] - curr_topic)))
#     print(np.sum(np.square(latent_topics - curr_topic), axis=1)[np.argsort(ssd)[index]])
#     print (ssd[np.argsort(ssd)[index]])
#     print (ssd[np.argsort(ssd)[index+1]])
#     print (ssd[np.argsort(ssd)[index-1]])
    print ('ssd', ssd[index])
    return np.argsort(ssd)[index]

# set to random, or you can set index to a number
index=random.randint(0,data.shape[0])
# index=13929
print(index)
print (data['text'][index])
similar_index = find_most_similar(index, all_latent_topics, data['text'])
print(similar_index)
print (data['text'][similar_index])


11882
That sport jacket totally matches your sneakers.
  
how many equal 0 1
ssd 0.188471
2151
Yes, that's a pistol drill in my pants, and yes, I'm glad to see you.
  


In [168]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn import linear_model

def guess_ratings(indices, ratings, joke_features):
    alpha = 9999999999
#     lasso =linear_model.LinearRegression()
#     lasso = Lasso(alpha=alpha, max_iter = 20000, tol=.1)
    lasso = linear_model.Ridge (alpha = .1)
#     lasso = ElasticNet(alpha=alpha, l1_ratio=0.7, max_iter=20000)
#     lasso

    y_pred_lasso = lasso.fit(joke_features[indices], ratings).predict(joke_features)
    y_pred_lasso -= np.mean(y_pred_lasso)
    y_pred_lasso *= 1.5/np.std(y_pred_lasso)
    y_pred_lasso += 2.5
    return y_pred_lasso
# add a joke's index and then its corresponding rating 
indices = [9539, 9943, 14327, 14328, 13048, 13058, 8698, 8701, 2578, 2598, 5497, 6235, 13886, 0, 15017, 13882, 10817, 13867, 14196, 13860, 13857, 10764, 13876, 10830, 10820, 13894, 13889, 13864, 13893, 14086, 13871, 13874, 12070, 13861, 13829, 13871, 12806, 12916, 14609, 14506, 14232, 13892, 13896, 13052, 13078, 4479, 10665, 3220, 5178, 4971, 10065, 14513, 4851, 14453, 14406, 14427, 221, 1399, 9912, 294, 9192, 130, 9960, 5156, 14276, 10745, 10175, 14434]
ratings = [4.5, 2, 3.3, 4, 5, 2, 3.8, 3, 2, 4, 3.5, 1, 3, 4, 3.5, 3, 0, 1, .5, 0, 2, 1, 0, 4, 4.5, 4.5, 2, 3, 3, 1, 2, 4, 1, 4, 2, 3, 4, 0, 4, 1, 2.1, 1, 2, 0, 0, 0, 5, 3, 4.5, 5, 1, 3.3, 3.4, 3, 3, 1, 0, 0, 0, 3, 2, 3, 1, 3, 2, 2, 2, 5]
indices, ratings = np.asarray(indices), np.asarray(ratings)
guesses = guess_ratings(indices, ratings, all_latent_topics)

# uncomment this to get "good jokes"
blah = random.randint(-100, -1)

# uncomment this to get "bad jokes"
# blah = random.randint(0, 1000)

# uncomment this to get a completely random joke
# blah = random.randint(0, 15000)


joke_index = np.argsort(guesses)[blah]
while joke_index in indices:
    blah -= 1
    joke_index = np.argsort(guesses)[blah]
# joke_index = random.randint(0, 15000)
print('index: ', joke_index)
print('rating: ', guesses[joke_index])
print(data['text'][joke_index])

index:  8129
rating:  6.78795546031
On a beautiful deserted island in the middle of nowhere, the following people are stranded:   2 Italian men and 1 Italian woman, 2 French men and 1 French woman, 2 German men and 1 German woman, 2 Greek men and 1 Greek woman, 2 English men and 1 English woman, 2 Polish men and 1 Polish woman, 2 Japanese men and 1 Japanese woman, 2 Australian men and 1 Australian woman, 2 New Zealander men and 1 New Zealander woman, 2 Irish men and 1 Irish woman, 2 American men and 1 American woman.   One month later, the following things have occurred....
 One Italian man killed the other Italian man for the Italian woman.  
 The two French men and the French woman are living happily together, having loads of sex. 
 The two German men have a strict weekly schedule of when they alternate with the German woman.  
 The two Greek men are sleeping with each other and the Greek woman is cleaning and cooking for them.  
 The two English men are waiting for someone to introd

In [863]:
print(data['text'][8278])    

It's gonna be so good it'll damage your hull. 
